### 可以看到以上的两种结构达到的都是大约92%的正确率，这并不是一个非常好的结果，我们可以通过改进网络的结构达到99%的正确率。
# 利用CNN搭建手写数字识别系统

### 1 载入MNIST数据

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../../proj2/第二天/练习/MNIST_data',one_hot=True)


/Users/liuyanfang/anaconda3/envs/tensorflow1.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting ../../proj2/第二天/练习/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../proj2/第二天/练习/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../proj2/第二天/练习/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../proj2/第二天/练习/MNIST_data/t10k-labels-idx1-ubyte.gz


## placeholder
### 2 我们从为输入的图像和输出的分类创建节点开始创建计算图

In [2]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

在此处x,y_ 都不是实际的值，当我们要求TensorFlow进行计算的时候我们会对对应的值进行输入。 此处x是一个浮点类型的2d tensor，我们定义时分配的shape 为[None, 784]，其中784是将28*28的MNIST图片变成一个向量，None表示第一维可以为任何大小，对应batch size。y_是每张图片对应的数字，也是一个2d tensor，其中第一维仍然可以任意大小，与batch size相等即可，每一行是一个one-hot 十维向量，每行只有一个值为1，图片中的数字的标签，其余9个位置都是0.

### 3 权重初始化 
建立模型的过程中我们会需要定义很多权重和偏置，初始化权重的时候应当增添随机的噪声。由于选用ReLU作为非线性激活函数，一般选取一个正值作为偏置的初值防止'dead neurons'的问题。下面定义两个函数完成上述功能。

In [3]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

### 4卷积和池化
TensorFlow提供了灵活的卷积和池化操作，在这个模型中我们将选择最简单的版本，卷积stride选择为1，用0填充，这样输出和输入的维度是相同的，池化操作是在2\*2的范围内选取最大值(max pooling)。

In [4]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### 第一个卷积层
我们现在可以实现我们的第一层了，其中包括了卷积层和池化层，卷积层将对每一个5\*5的patch计算32个feature，因此它的权重的维度为[5, 5, 1, 32]，其中前两维表示了patch 的大小，第三个数字表示输入的图片通道数，32表示输出的通道数。对每一个通道的输出我们还会有偏置部分。

In [5]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

在数据输入模型之前，我们需要先对输入数据的维度进行更改，变为一个4维张量，第2，3，4个维度分别表示图像的宽，高和输入图像的颜色通道数，第一维的-1表示这一个维度是通过计算从而保证整个数据大小不发生改变。

In [6]:
x_image = tf.reshape(x, [-1, 28, 28, 1])

当模型对输入图像进行卷积的时候，我们先使用权重与图像进行卷积，然后加上偏置向量，最后经过非线性激活函数ReLU，最后再经过池化，经过池化以后图像的大小将会变成14*14。

In [7]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### 第二层
与第一层类似，输出为64个特征。


In [8]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### 全连接层
经过两层卷积和池化以后，图像的大小已经变成了7\*7，我们现在增加一个含有1024个神经元的全连接层来对模型进行处理。首先将图像从二维转化为一维，然后与权重相乘，加上偏置后通过激活函数。
$h_{fc1} = ReLU(h W_{fc1} + b_{fc1})$

In [9]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Dropout
Dropout可以避免模型过拟合，我们在模型的输出之前使用dropout，创建一个placeholder表明神经元保留的概率，这可以使我们在训练的时候使用dropout，在测试的时候不适用dropout

In [10]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### 输出层

输出层为全连接层，与softmax回归模型类似。

In [11]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

### 训练和评估模型
基本步骤与前面相同，主要的区别在于：
* 使用ADAM优化算法替代梯度下降
* 增加了keep_prob参数来控制dropout的概率

In [12]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.06
step 100, training accuracy 0.82
step 200, training accuracy 0.86
step 300, training accuracy 0.92
step 400, training accuracy 0.92
step 500, training accuracy 0.96
step 600, training accuracy 0.9
step 700, training accuracy 0.88
step 800, training accuracy 0.92
step 900, training accuracy 0.96
step 1000, training accuracy 0.9
step 1100, training accuracy 0.94
step 1200, training accuracy 0.92
step 1300, training accuracy 0.98
step 1400, training accuracy 0.94
step 1500, training accuracy 1
step 1600, training accuracy 0.94
step 1700, training accuracy 1
step 1800, training accuracy 1
step 1900, training accuracy 0.98
test accuracy 0.9732
